### LSTM

In [1]:
from cryptocmd import CmcScraper
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, TimeDistributed
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import numpy as np

In [6]:
scraper = CmcScraper("BTC")
headers, data =  scraper.get_data()
bit_data = pd.DataFrame(data, columns=headers)
bit_data.to_csv("bit_data.csv", index=False)

In [2]:
# Univariate LSTM - Predice the next possible value

# Sample data

items = [10,20,30,40,50,60,70,80,90,100,110,120,130]

def make_sequence(data, n_steps):
    X,y = [], []
    for i in range(len(data)): 
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps])
        if(i + n_steps == len(data) -1 ):
            break
    return X,y

data_lstm = make_sequence(items, 3)
X = np.array(data_lstm[0])
X = X.reshape(X.shape[0], X.shape[1], 1)

In [10]:
# Vanilla LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation="relu", input_shape=(3, 1)))
lstm_model.add(Dense(1))

lstm_model.compile(optimizer="adam", loss="mse")
lstm_model.fit(X, np.array(data_lstm[1]), epochs=200, verbose=0) 

In [11]:
# Stacked LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation="relu", input_shape=(3, 1), return_sequences=True))
lstm_model.add(LSTM(50, activation="relu"))
lstm_model.add(Dense(1))

lstm_model.compile(optimizer="adam", loss="mse")
lstm_model.fit(X, np.array(data_lstm[1]), epochs=200, verbose=0) 

In [13]:
# Bidirectional Modle - The model can learn the input sequences both forward and backward

model = Sequential()
model.add(Bidirectional(LSTM(50, activation="relu"), input_shape=(3,1)))
model.add(Dense(1))

model.compile(loss='mse', optimizer="adam")
model.fit(X, np.array(data_lstm[1]), epochs=200, verbose=0)

In [5]:
#predictions
input_data = np.array([110,120,130])
input_data = input_data.reshape(1,3,1)

In [15]:
lstm_model.predict(input_data)

1/1 [==============================] - 0s 278ms/step


array([[140.63837]], dtype=float32)

In [16]:
stacked_model.predict(input_data)

1/1 [==============================] - 0s 257ms/step


array([[140.76653]], dtype=float32)

In [17]:
model.predict(input_data)

1/1 [==============================] - 0s 255ms/step


array([[140.94589]], dtype=float32)

### LSTM CNN

In [6]:
items1 = [10,20,30,40,50,60,70,80,90, 100, 110,120]
data_lstm1 = make_sequence(items1, 4)

X1 = np.array(data_lstm1[0])
X1 = X1.reshape(X1.shape[0], 2, 2, 1)

In [7]:
conv1d_model = Sequential()
conv1d_model.add(TimeDistributed(Conv1D(filters=64, activation="relu", kernel_size=1), input_shape=(None, 2, 1)))
conv1d_model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
conv1d_model.add(TimeDistributed(Flatten()))

conv1d_model.add(LSTM(50, activation="relu"))
conv1d_model.add(Dense(1))

conv1d_model.compile(loss='mse', optimizer="adam")
conv1d_model.fit(X1, np.array(data_lstm1[1]), epochs=500, verbose=0)

In [11]:
input_data1 = np.array([100, 110,120,130])
input_data1 = input_data1.reshape(1,2, 2, 1)

In [13]:
conv1d_model.predict(input_data1)

1/1 [==============================] - 0s 239ms/step


array([[141.3129]], dtype=float32)

### Multivariate LSTM

In [14]:
seq_1 = np.array([10,20,30,40,50,60,70,80,90])
seq_2 = np.array([15,25,35,45,55,65,75,85,95])
seq_3 = np.array([seq_1[i] + seq_2[i] for i in range(len(seq_1))])

In [17]:
seq_1, seq_2, seq_3

(array([[10],
        [20],
        [30],
        [40],
        [50],
        [60],
        [70],
        [80],
        [90]]),
 array([[15],
        [25],
        [35],
        [45],
        [55],
        [65],
        [75],
        [85],
        [95]]),
 array([[ 25],
        [ 45],
        [ 65],
        [ 85],
        [105],
        [125],
        [145],
        [165],
        [185]]))

In [16]:
seq_1, seq_2, seq_3 = seq_1.reshape(9,1),seq_2.reshape(9,1),seq_3.reshape(9,1)

In [18]:
data = np.hstack((seq_1, seq_2, seq_3))

In [20]:
data.shape

(9, 3)

In [38]:
def new_seq(data, n_steps):
    X,y = [], []
    for i in range(data.shape[0]):
        X.append(data[i:i + n_steps, :-1])
        y.append(data[i + n_steps - 1,-1])
        if i + n_steps == data.shape[0] - 1:
            break
        

    return X,y

X, y = np.array(new_seq(data, 3)[0]), np.array(new_seq(data, 3)[1])


In [39]:
X.shape, y.shape

((6, 3, 2), (6,))

In [37]:
data[0:3, :-1], data[2,-1]

(array([[10, 15],
        [20, 25],
        [30, 35]]),
 65)